In [1]:
comment = 'calculated in Plots_ExpClass.ipynb'

In [2]:
%matplotlib inline
#%matplotlib notebook
import os
import pandas as pd
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
from Datasets import init_generator # returns HigherOrderPathGenerator

In [4]:
gen_school = init_generator('primaryschool_1.csv')

97223 rules read
[]
[]


In [5]:
if not os.path.exists('tmp'):
    os.makedirs('tmp')

# Check child-teacher-child interactions
Figure 4.11

In [6]:
def is_teacher(node):
    return gen_school._target_node_metadata['Role'][node] == 'Teacher'
def is_child(node):
    return gen_school._target_node_metadata['Role'][node] == 'Child'
def get_class(node):
    return gen_school._target_node_metadata['Class'][node]

In [7]:
def get_class_class_probs_via_teacher(HON=True, title=None, filename=None, transition_probs=False, figsize=(7,5), dpi=200, bbox_inches='tight', **kwargs):
    """
    returns a dictionary { (c1,c2) : prob}
    where (c1,c2) is a pair of classes
    and prob = P(class(n1) = c1, class(n3)=c2 | is_child(n1), is_teacher(n2), is_child(n3))
    where (n1,n2,n3) is a random path of length 3 starting with n1 ~ stationary
    """
    gen = gen_school if HON else gen_school.to_FON()
    total_prob = 0
    cp = defaultdict(float) # prob of class
    ccp = defaultdict(float) # prob of pair of classes
    for (n1,n2,n3),prob in gen.path_probs(start=(), num_steps=3):
        if is_child(n1) and is_teacher(n2) and is_child(n3):
            c1 = get_class(n1)
            c3 = get_class(n3)
            total_prob += prob
            cp[c1] += prob
            ccp[(c1,c3)] += prob
    print('total probability', total_prob)
    if transition_probs:
        ccp = { cc:prob/cp[cc[0]] for cc,prob in ccp.items()} # transition probability c1->c2
    else:
        ccp = { cc:prob/total_prob for cc,prob in ccp.items()} # probability of pairs
    res = pd.Series(ccp).unstack()
    if filename is not None:
        fig = plt.figure(figsize=figsize, dpi=dpi)
        if title is not None:
            fig.suptitle(title)
        classes = sorted(cp.keys())
        ax = fig.gca()
        sns.heatmap(res, annot=True, fmt='.2f', cmap=plt.cm.Blues, xticklabels=classes, yticklabels=classes, ax=ax)
        if transition_probs:
            ax.set_xlabel('target')
            ax.set_ylabel('source')
        else:
            ax.set_xlabel('class of $child_2$')
            ax.set_ylabel('class of $child_1$')
        fig.savefig(filename, bbox_inches=bbox_inches, **kwargs)
        with open(filename + '.txt','w') as f:
            gen.write_config(f, comment=comment, prefix='', sep='\t')
            f.write('\n\nProbability of a child->teacher->child path %f\n' % total_prob)
            if transition_probs:
                f.write('\nTransition probabilities:\n')
            else:
                f.write('\nPair probabilities:\n')
            f.write(res.to_csv(line_terminator='\n'))
    return res

In [ ]:
title_HON = 'Cond. probability of Child-Teacher-Child interaction by class (HON)'
c2c_HON = get_class_class_probs_via_teacher(True, title_HON, filename='tmp/school_hon_interaction_via_teacher.png')

In [ ]:
title_FON = 'Cond. probability of Child-Teacher-Child interaction by class (FON)'
c2c_FON = get_class_class_probs_via_teacher(False, title_FON, filename='tmp/school_fon_interaction_via_teacher.png')

Get frequencies by class from stationary distribution

In [10]:
class_probs = defaultdict(float)
for _,n,prob in gen_school.transition_probs(start=()): # stationary distribution
    class_probs[get_class(n)] += prob
class_probs

defaultdict(float,
            {'5B': 0.08779785547204058,
             '5A': 0.04456183510387105,
             '4A': 0.04474650478145715,
             'Teachers': 0.004609362345706892,
             '3B': 0.15727510236331413,
             '4B': 0.05062492826777912,
             '2A': 0.11235273025299394,
             '1B': 0.2166310296376714,
             '2B': 0.12849865639303715,
             '1A': 0.04554663582826103,
             '3A': 0.10735535955386742})